<a href="https://colab.research.google.com/github/nancy-kataria/NexTrade/blob/main/product_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
path = kagglehub.dataset_download("vivek468/superstore-dataset-final")
csv_file_path = path + "/Sample - Superstore.csv"
superstore_data = pd.read_csv(csv_file_path, encoding='ISO-8859-1')
superstore_data.head()

100%|██████████| 550k/550k [00:00<00:00, 64.4MB/s]

Extracting files...


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
superstore_data.isnull().sum()

,0
Row ID,0
Order ID,0
Order Date,0
Ship Date,0
Ship Mode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0


Collaborative Filtering–based Product Recommendation

In [5]:
# Create a user-product interaction matrix
user_product_matrix = superstore_data.pivot_table(
    index='Customer ID',
    columns='Product ID',
    values='Quantity',  # or 'Sales'
    aggfunc='sum'
).fillna(0)


product_similarity = cosine_similarity(user_product_matrix.T)
product_similarity_df = pd.DataFrame(
    product_similarity,
    index=user_product_matrix.columns,
    columns=user_product_matrix.columns
)

In [7]:
def recommend_products(product_id, similarity_df, top_n=5):
    similar_scores = similarity_df[product_id].sort_values(ascending=False)
    return similar_scores[1:top_n+1]

recommend_products('FUR-CH-10000454', product_similarity_df)

,FUR-CH-10000454
Product ID,
TEC-PH-10000215,0.393190
OFF-PA-10002245,0.348398
FUR-CH-10004495,0.341867
FUR-FU-10003424,0.331083
TEC-PH-10004348,0.296762
